In [110]:
import json
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import re


In [72]:
path = '/Users/elisilvert/Desktop/healthchain/fhir/'


In [73]:
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
print(len(onlyfiles))

1180


In [111]:
def process_dem_data(el):
    el1 = el['resource']

    pid = el1['id']
    extension = el1['extension']
    name = el1['name']
    telecom = el1['telecom']

    #store this
    race = ''
    ethnicity =''
    birthplace=''
    disability_adjusted_life_years=''
    quality_adjusted_life_years=''
    for d in extension:
        if "us-core-race" in d['url']:
            race = d['extension'][-1]['valueString']
        if "us-core-ethnicity" in d['url']:
            ethnicity = d['extension'][-1]['valueString'] 
        if "patient-birthPlace" in d['url']:
            birthplace = d['valueAddress']
        if "disability-adjusted-life-years" in d['url']:
            disability_adjusted_life_years = d['valueDecimal']
        if "quality-adjusted-life-years" in d['url']:
            quality_adjusted_life_years = d['valueDecimal']

    firstname = ''
    lastname = ''
    for d in name:
        if "official" in d['use']:
            lastname = d['family']
            firstname = d['given'][0]
            lastname = re.sub(r'\d+', '', lastname)
            firstname = re.sub(r'\d+', '', firstname)
            
    phone = ''
    for d in telecom:
        if 'phone' in d['system']:
            phone = d['value']

    gender = el1['gender']
    birthdate = el1['birthDate']
    line = el1['address'][0]['line'][0]
    city = el1['address'][0]['city']
    state = el1['address'][0]['state']
#             postalcode = el1['address'][0]['postalCode']
    country = el1['address'][0]['country']
    address = ', '.join([line,city,state])
    demographics = {'patient_id':pid,
                    'first_name':firstname,
                    'last_name':lastname,
                    'gender':gender,
                    'birthdate':birthdate,
                    'phone':phone,
                    'address':address,
                    'race':race,
                    'ethnicity':ethnicity,
#                     'birthplace':birthplace,
                   }  
    return demographics

In [118]:
def clean_fhir_data(file):
    
    f = open(path+file)
  
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    data1 = data['entry']
    cleaned_data1 = {}
    conditions_l = []
    procedures_l = []
    medications_l = []
    for el in data1:
        resource_type = el['resource']['resourceType']
#         l.append([resource_type])
        if resource_type == 'Patient':
            cleaned_data1.update(process_dem_data(el))
        if resource_type == 'Condition':
            conditions_l.append(el['resource']['code']['text'])
        if resource_type == 'Procedure':
            procedures_l.append(el['resource']['code']['text'])
        if resource_type == 'MedicationRequest':
            medications_l.append(el['resource']['medicationCodeableConcept']['text'])
            
#             return el
    cleaned_data1['conditions'] = list(set(conditions_l))
    cleaned_data1['procedures'] = list(set(procedures_l))
    cleaned_data1['medications'] = list(set(medications_l))
    return cleaned_data1
    
l = []
for file in onlyfiles:
    l.append(clean_fhir_data(file))

In [120]:
print(len(l))

1180


In [121]:
# Serializing json
json_object = json.dumps(l, indent=4)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)